In [1]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
from sys import exc_info
from caiman import movie as cmovie
import json
import pickle

def tryexcept(x,default):
    try:
        return eval(x)
    except:
        return default

In [2]:
# autoreload setup
%load_ext autoreload
%autoreload 1
%aimport Regions1, physio_def_1, numeric, Automatic1

from physio_def_1 import *
from Regions1 import Regions
from numeric import *
from Automatic1 import *

In [37]:
# global vars
if "srdjan" not in os.getcwd():
    os.chdir(os.path.expanduser("~"))

if "srdjan" in os.getcwd():
    exampleNpzFile = "../test_data/8gl.npz"
#     exampleNpzFile = "/data/Sandra/2020_05_13/Experiment48b/Experiment48b_0_Series009-11.npz"
else:
    exampleNpzFile = "/data/Sandra/2020_05_13/Experiment48b/Experiment48b_0_Series009-11.npz"

outputStyle = {
    "color":"blue",
    "font-family":"Courier New",
    "font-size":"90%",
    }
infoStyle = {
    "font-size":"80%",
    "color":"grey",
    }

baseFig = getFigure(w=200,h=150)

## global vars
test = "srdjan" in os.getcwd()
debug = "srdjan" in os.getcwd()

if test:
    %config InlineBackend.figure_format = 'retina'

In [206]:
try: del regions0
except: pass
try: del regions
except: pass

In [207]:
timescales = sorted([i*j for i in [1,2,3,5,] for j in [.1,1,10]])[:-1]+ sorted([i*j*60 for i in [1,1.5,2,3,5,10,30] for j in [1]])
timescales = np.array(timescales)

# tspositions = (np.log(timescales)*4).astype(int)
# tspositions -= tspositions[0]
tspositions = np.arange(len(timescales))
# plt.plot(tspositions,".-")
# mld = {.2:"ultrafast",3:"fast",600:"slow"}
# msd = {.2:"red",3:"black",600:"grey"}

# labels = {.2:"ultrafast",3:"fast",600:"slow"}

# sliderMarkers = {p:{"label":labels[ts], "style":{"color":"red"}} for p,ts in zip(tspositions,timescales) if ts in labels }
# sliderMarkers

In [208]:
app = JupyterDash(__name__,
                  width=1000,
                  height=1000,
                 )

APP_LAYOUT = [
    
    html.H1(children='CaRec Analyzer'),
    html.Div("Please, think of a better name and let me know. :-)",style=infoStyle),
    html.Br(),
    
    html.H3("Import recording"),################################################################################
    
    html.Div('Enter path to a file and press enter.',style={ "display":"inline-box"}),
#     html.Div('If you I will not check for movement, so if you already have a motion-corrected file, input that directly.',style={**infoStyle, "display":"inline-box"}),
    
    dcc.Input(id="filename_input",
            type="text",
            placeholder=exampleNpzFile,
            debounce=True,
            size = 60,
#             value=exampleRoiFile if test else "",
            value=exampleNpzFile if test else "",
        ),

    html.Br(),
    
    html.Div(id="filename_feedback", children="",style=outputStyle),#To start, enter the full path to a file and press enter. Note, that loading can take a few seconds [for very large files even a minute].
    
    html.Div([html.Br(),"Re(set) Frequency to "], style={'display': 'inline-block'}),
    dcc.Input(id="freq_input",
        type="text",
        debounce=True,
        size = 5,
        value="",
        placeholder="3",
        style={'display': 'inline-block', "margin-left":"5px", "margin-right":"5px", "width":"40px"}),    
    html.Div("Hz.",style={'display': 'inline-block'}),
    html.Button("(Re)set frequency", n_clicks=0, id="freq_button",style={"margin-left":"5px"}),
    html.Br(),
    html.Div("[Only if you need override automatic setting, or in the absence of metadata]",
             style={**infoStyle,'display': 'inline-block', "margin-left":"5px"}),

    html.Div(id="freq_feedback", children="",style=outputStyle),
    
    html.H3("Cell detection"),################################################################################
    
    html.Div(["Use the slider if you need to select only a part of the movie."],style={'display': 'inline-block'}),
    html.Div("For example, if there is a single distinct movement in the movie, you may want to analyze the two parts separately.", style=infoStyle),

    html.Div(children=[
        html.Pre(
        id="relay_out",
        children="relay_out",
        style={
            "vertical-align":"text-top",
            "width"    : "200px",
            "height"   : "50px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
              })
    ],style={"display":"inline-block" if debug else 'none'}),
    
    html.Br(),
    html.Div(dcc.Graph(id='FOV_trace',     figure=baseFig), style={"display":"inline-block"}),
    html.Br(),
    
    html.Div(children="Spatial filtering", style={'display': 'inline-block'}),
    dcc.Input(id="gSig_filt_input",
        type="text",
        debounce=True,
        size = 5,
        value=tryexcept("str(regions.filterSize)",""),
        placeholder="3",
        style={"width":"35px", "margin-right":"5px", "margin-left":"5px"}),    

    html.Div(html.Abbr(
        children="[info]",
        title=
        """    For the proper analysis, you need to set the size of the spatial filter. Its value should be of the order of the  half of the typical cell dimension (in pixels!). 
    You can also combine filtering at different scales by inputing a list of two (or more) integers. For example, '[5,3]' will combine filtering at 5 and at 3. Sometimes this capures better the finer structure."""
        ),style={**infoStyle,'display': 'inline-block', "margin-right":"35px"}),
    
    html.Button("Calculate ROIs", n_clicks=1 if  debug else 0, id="rois_button"),
    html.Div(id="rois_feedback", children="",style=outputStyle),

    html.Br(),
    
    html.Div([
        dcc.Graph(id='roi_selector', figure=baseFig),
        
        html.Div([
           "Selected ROIs:",
            dcc.Input(id="selected_indices",
                type="text",
                debounce=True,
                size=6,
                value="",
             ), 
        ],style={"display": "inline-box" if debug else "hidden","width":"100"}),
        html.Button('Discard unselected', id='discard_button', style={"display":"inline-box"},
                    n_clicks=1),
        html.Div(id="discard_feedback",children="",
             style={"display":"inline-box",**outputStyle,}
            ),
    ], style={"display":"inline-block"}),
    html.Div([
        dcc.Graph(id='show_hover_trace',figure=baseFig),
        html.Div([
            html.Div("Spike filter",style={"margin-left":"40px", "display":"inline-block"}),
            dcc.Input(id="filter_slider_feedback",
                      value="",
#                       size = 20,
                     style={
                        "margin-right":"5px",
                        "margin-left":"5px",
                        "display":"inline-block" if debug else 'none',
                         "width":"60px"
#                         'border' :'thin lightgrey solid',
                     }),
            dcc.Slider(
                id = "filter_slider",
                min=tspositions[0],
                max=tspositions[-1],
                value=tspositions[7],
                    marks = {
                         1:    {'label': 'ultrafast', 'style': {'color': 'red'}}, 
                         7:    {'label': 'fast', 'style': {'color': 'black'}}, 
                        16:    {'label': 'slow', 'style': {'color': 'grey'}}
                            },
#                 step=1,
                updatemode='drag',
#                 included=False
                ),
            html.Div("Smooth",style={"margin-left":"40px", "display":"inline-block"}),
            dcc.Slider(
                id = "smooth_slider",
                min=0,
                max=20,
                value=0,
                step=1,
                updatemode='mouseup',
#                 included=False
                )
        ], 
            style={"width":"400px", "display":"inline-block",
#                    "border":"thin lightgrey solid", 
                  }),
    ], style={
#         "width":"410px",
        "display":"inline-block", 
#         "border":"thin lightgrey solid",
             }),
    html.Br(),
    
    html.Div(children=[
        html.Pre(
        id="selected_out",
        children="selected_out",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll'
              }),
        html.Pre(
        id="hover_out",
        children="hover_out",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
              })
    ],style={"display"  : "inline-block" if debug else "none"}),
    
    
    html.Div(children=
        html.Pre(
        id="spare_out",
        children="spare",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
            "display"  : "inline-box"
              }),
             style={"display":"inline-block" if debug else "none"}),
    
    dcc.Graph(id='raster',figure=baseFig),
    html.H3("Save"),
    html.Div([
        "When you are happy with your analysis, you can save it. By default, the analysis (traces and coordinates) is saved as the current date (e.g. 2020_02_18) in the directory",
        html.Div("", style=outputStyle, id="analysis_folder"),
        "You can modify the filename by adding to it some other identifier(s) by entering them in the input box below. I suggest your username.",
        "When you are ready, press save."
        "Note also that the traces and times will be saved with the frequency as shown in plots."
    ]),
    dcc.Input(id="save_file_append",
        type="text",
        debounce=False,
        size = 30,
        placeholder="e.g. username",
        value='',style={"display":"inline-box"}),
    html.Button('Save', id='save_button', style={"display":"inline-box"},
                n_clicks=0
               ),
    html.Div(id="save_feedback",
             style={"display":"inline-box",**outputStyle}
            )
    ]

In [209]:
# @app.callback(Output('filter_slider_feedback', 'children'),
#               [Input('filter_slider', 'value')])
@app.callback(Output('filter_slider_feedback', 'value'),
              [Input('filter_slider', 'value')])
def display_value(value):
    t = timescales[value]
    
    if t<1:
        tshow = t*1000
        unit = "ms"
    elif t>60:
        tshow = t/60
        unit = "min"
    else:
        tshow = t
        unit = "s"
    return f'{tshow:.3g} {unit}'


In [210]:
def loadNpzFiles(val, dryrun=False, reimport=True):
    from pandas import read_csv
    npzFile = val
    analysisFolder = npzFile.replace(".npz","_analysis")
    feedback = []
    if reimport:
        if "Series" in npzFile:
            serrange = npzFile.rstrip(".npz").split("Series")[1].split("-")
            serrange = [int(el) for el in serrange]
            singleFile = len(serrange)==1
        else: singleFile = True
        if singleFile:
            if dryrun:
                movie = np.zeros((100,10,10))
            else:
                movie = import_npz_files([npzFile])
            feedback += [
                f"{npzFile} loaded successfully.",
                html.Br(),
            ]
        else:
            if len(serrange)>2:
                feedback += ["Expecting only two numbers (beginning and end), but got more than that. Proceeding with the first and the last. "]
            serrange = range(serrange[0],serrange[-1]+1)
            ianchor = int(os.path.split(val)[1].split("_Series")[0].split("_")[-1])
            npzFiles = [(npzFile.split("Series")[0]+"Series%03i.npz"%i).replace(f"_{ianchor}_", f"_{ianchor+ia}_") for ia,i in enumerate(serrange)]
            if dryrun:
                movie = np.zeros((100,10,10))
            else:
                movie = import_npz_files(npzFiles)
            for f in npzFiles:
                feedback += [
                    f"{f} loaded successfully.",
                    html.Br(),
                ]
        movie = movie.astype("float")
        if len(movie.shape)==2:        
            movie = movie.reshape((1,-1))
        movie = cmovie(movie)

    try:
        if singleFile:
            metadata = read_csv(npzFile.replace("npz","txt")).loc[0]
        else:
            feedback += ["Assuming the metadata for all recording are the same, and importing it from the first recording only.",
                    html.Br(),]
            metadata = read_csv(npzFiles[0].replace("npz","txt")).loc[0]
        movie.fr = metadata.freq
        timeframes = metadata['T']
        if singleFile: assert timeframes==len(movie)
        freq = metadata.freq
        pxsize = metadata.pxSize
        feedback += [
            f"Info from metadata: frequency {freq:.4} Hz, and pixel size {pxsize:.3} µm. ",
            f"Movie shape is {movie.shape}. "
        ]
        if pxsize<.8:
            feedback += [
            f"This is too tiny pixel, so I will decrease the resolution to increase the pixel size twice to {(2*pxsize):.3} µm. ",
            ]
            movie = rebin(rebin(movie, 2, axis=1, norm=False), 2, axis=2, norm=False)
            pxsize *= 2
        outFilterSize = str(int(np.ceil(6./pxsize)))
        feedback += [
            html.Br(),
            f"Automatic size of the spatial filter based on the pixel size: {outFilterSize} [you can change it below] ",
            html.Br(),]
    except:
        feedback += [
            f"WARNING: Missing medatada. I will assume a frequncy of 1Hz for the first setup.",
            "In case you actually know the frequency, you can change it down."
            "Also, think carefully about the size of the spatial filter. See hover info.",
        html.Br(),
        ]
        outFilterSize = None
        freq = 1

    movie.fr = freq


#     # resample temporarily to lower frequency, to initiate ROIs and check for motion correction
#     if freq>3:

#         freqMC = 3
#         n_rebin = int(movie.fr/freqMC)

#         rebinned_movie = rebin(movie,n_rebin).astype("float32")
#         rebinned_movie = cmovie(rebinned_movie)
#         rebinned_movie.fr = movie.fr/n_rebin
#     else:
#         rebinned_movie = movie.astype("float32")

    # initiate ROIs
    regions = Regions(movie, full=False, diag=True)
    del regions.statImages["diff_mean"]
    regions.movie = movie
    regions.file = val
    regions.trange = regions.time[0], regions.time[-1]
    regions.analysisFolder = analysisFolder
    try: regions.metadata = metadata
    except: pass

    return (feedback, outFilterSize, regions)

In [211]:
######################################################################################
@app.callback([Output("filename_feedback", "children"),
               Output("gSig_filt_input", "value"),
               Output("freq_input", "value"),
               Output("analysis_folder","children"),
#                Output("roi_selector0", "figure"),
               Output("FOV_trace", "figure")
              ],
              [Input("filename_input", "value")],
#               [State("check_movement_choice","value")]
             )
def loadFiles_callback(val):
    if str(val)=="":
        return ('Loading can take some time [for very large files even minutes].',"","","",baseFig)
    try:
        global regions
        if test:
            try:
                regions.movie
                reimport = False
            except: reimport = True
        else:
            reimport = True
        print (reimport)
        if val.endswith(".npz"):
            if reimport:
                feedback, filterOut, regions = loadNpzFiles(val, reimport=reimport)
            else:
                filterOut = str(regions.filterSize)
                feedback = "Testing. Did not reimport."
            freqOut, analysisFolder = regions.movie.fr, regions.analysisFolder
        
        if val.endswith(".roi") or val.endswith(".pkl"):
            return (["Sorry, importing pickles is not yet supported."], "", "", "",baseFig)
        
        regions.get_fov_trace()
        fovTrace = fov_trace(regions, twoRows=False) 
        
        return (feedback, filterOut, freqOut, analysisFolder,fovTrace)
            
    except:
        return ([html.P("ERROR: "+exc_info().__repr__())], "", "", "", baseFig)


In [212]:
######################################################################################
@app.callback(
    Output("freq_feedback", "children"),
    [Input("freq_button", "n_clicks")],
    [State("freq_input", "value")]
             )
def resetFreq_callback(n_clicks,newfreq):

    try:
        if n_clicks <= 0:
            return ""
        else:
            global regions
            newFreq = float(newfreq)
            regions.time = np.arange(len(regions.movie))/newFreq
            regions.movie.fr = newFreq
            return "Movie frequency set to %.1f Hz"%newFreq
    except:
            return "ERROR: "+exc_info().__repr__()

In [213]:
@app.callback([
    Output("rois_feedback", "children"),
    Output("roi_selector","figure"),
    ],
    [
    Input("rois_button", "n_clicks"),
    Input("discard_button", "n_clicks"),
    ],
    [
    State("gSig_filt_input", "value"),
    State("relay_out", "children"),
    ]
             )
def rois_callback(n_clicks_rois, n_clicks_discard, spFilt,relayOut):
    feedback = []
    try:
        from dash import callback_context as ctx

        if not ctx.triggered:
            button_id = 'No clicks yet'
            return "", baseFig
        
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if debug:
            feed
        
        if len(spFilt)==0:
            return "Spatial filter needs to be an integer or an itarable of integers.", baseFig
        
        global regions
        
        feedback = []
        if "rois" in button_id:
            x = eval(spFilt)
            if type(x) == int:
                x = [x]

            update = 0

            currtrange = np.array(list(regions.trange))
            if len(relayOut) and relayOut!="relay_out":
                newtrange = np.array(list(eval(relayOut)["xaxis.range"]))
                dt = np.diff(regions.time[:2])[0]
                for j in [0,1]:
                    if np.abs(newtrange[j]-currtrange[j])>dt:
                        update += 1
            if update:
                feedback += [f"Time range changed: {currtrange} -> {newtrange}", html.Br()]
                t_begin, t_end = newtrange
            else:
                t_begin, t_end = currtrange

            try:
                regions.filterSize
                if sorted(regions.filterSize) != sorted(x):
                    update += 1
                    feedback += [f"Filter has changed to {x}.",html.Br()]
            except:
                update += 1
                feedback += [f"Filter set to {x}.",html.Br()]


            if update:
                time = np.arange(len(regions.movie))/regions.movie.fr
                i_begin = np.where(time>=t_begin)[0][0]
                i_end = np.where(time<=t_end)[0][-1]
    #             movie = rebin(regions.movie[i_begin:i_end], time_resample)
    #             movie.fr = regions.movie.fr/time_resample
                analysisFolder = regions.analysisFolder
                regions = createRegions(regions.movie, gSig_filt=x,diag=True, FrameRange = (i_begin, i_end))
    #             regions.time = np.arange(len(movie))/movie.fr
                regions.trange = regions.time[0], regions.time[-1] 
                regions.analysisFolder = analysisFolder
                # merge obvious ROIs
                from Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph
                while True:
                    peak2bnd = getPeak2BoundaryDF(regions.df)
                    df = peak2bnd.query("dist<1.")[["i","j"]]
                    if len(df)==0: break
                    gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=False)
                    dropped = mergeBasedOnGraph(gRois,regions)
                    if dropped == 0: break

                regions.purge_lones((min(x)*.5)**2)
                regions.sortFromCenter()
        im = regions.statImages["diff_std"]
        feedback += [f" Number of ROIs: {len(regions.df)}"]
        roisImage = showRoisOnly(regions,indices=regions.df.index,im=im)
        return feedback, roisImage

    except:
        feedback+=["ERROR: "+exc_info().__repr__(),str((n_clicks_rois, n_clicks_discard,spFilt,relayOut))]
        return feedback, baseFig
    

In [218]:
rois_callback(1,1,"p","")

(["ERROR: (<class 'dash.exceptions.MissingCallbackContextException'>, MissingCallbackContextException('dash.callback_context.triggered is only available from a callback!'), <traceback object at 0x7f2128891050>)",
  "(1, 1, 'p', '')"],
 Figure({
     'data': [],
     'layout': {'height': 150,
                'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0},
                'plot_bgcolor': 'lightgrey',
                'template': '...',
                'width': 200,
                'xaxis': {'range': [0, 1], 'tickvals': []},
                'yaxis': {'range': [0, 1], 'tickvals': []}}
 }))

In [214]:
@app.callback(
    Output("selected_out", "children"),
    [Input("roi_selector", "selectedData")],
    )
def showSelected(selData):
    try:
        if selData is None:
            return "selected_out"
        return json.dumps(selData, indent=2)
    except:
        return "ERROR from showSelected: "+exc_info().__repr__() 
    
@app.callback(
    Output("hover_out", "children"),
    [Input("roi_selector", "hoverData")],
    )
def showHovered(hoverData):
    try:
        if hoverData is None:
            return "hover_out"
        return json.dumps(hoverData, indent=2)
    except:
        return "ERROR from showHovered: "+exc_info().__repr__()


@app.callback(
    Output("selected_indices", "value"),
    [Input("roi_selector", "selectedData")]
)
def collectSelected(selData):
    try:
        out = np.unique([pt["hovertext"] for pt in selData["points"]])
        out = sorted(out,key=int)
        return ", ".join(out)
    except:
        return "" 


@app.callback(
#      Output("roi_selector", "figure"),
     Output("discard_feedback", "children"),
    [Input("discard_button",   "n_clicks")],
    [State("selected_indices", "value")]
             )
def discard_callback(n_clicks,selected):
    try:
        if n_clicks <= 0:
            return ""

        global regions
        if selected == "":
            out = "nothing selected." if debug else ""
        else:
            selectedIndices = np.array(list(eval(selected)))
            nremoved = len(regions.df)-len(selectedIndices)
            regions.df = regions.df.loc[selectedIndices]
            out = "%i rois removed. Please, press again the button to 'Calculate Rois', to replot the remaining rois."%(nremoved)

#         roisImage = showRoisOnly(regions,indices=regions.df.index,im=regions.statImages["diff_std"])
#         if narrowHover:
#             roisImage.update_layout({"width":220,"height":240})
        return out
    except:
        return "ERROR: "+exc_info().__repr__()

In [215]:
app.layout = html.Div(children=APP_LAYOUT, style={"font-family":"Arial"})
app

In [216]:

######################################################################################
@app.callback(
    Output("relay_out", "children"),
    [Input("FOV_trace", "relayoutData")],
    )
def showRelayoutData(relData):
    try:
        if relData is None:
            return "relay_out"
        return json.dumps(relData, indent=2)
    except:
        return "ERROR from showRelayoutData: "+exc_info().__repr__() 

In [160]:
@app.callback(
    Output("show_hover_trace", "figure"),
    [Input("roi_selector", "hoverData"),
     Input("roi_selector", "selectedData"),
     Input("filter_slider_feedback", "value"),
     Input("smooth_slider", "value")
    ])
def plotHovered_callback(hoverData, selData, filterScale, smooth=0):
    from dash import no_update
    try:
        global regions
        try:
            hoverData.dtype
            ix = hoverData
        except:
            try:
                ix = [int(hoverData["points"][0]["hovertext"])]
            except:
                ix = []
        try:
            selData = np.unique([int(pt["hovertext"]) for pt in selData["points"]])
            ix = np.unique(np.hstack([selData, ix]))
        except:
            pass
        if len(ix)>20:
            return no_update
        timeScale, unit = filterScale.split()[:2]
        timeScale = float(timeScale)
        if unit=="min":
            timeScale *= 60
        if unit=="ms":
            timeScale /= 1000
        showCol = "zScore_%g"%timeScale
        try:
            regions.df[showCol]
        except:
            regions.fast_filter_traces(timeScale)
        offset = 10/(2*smooth+1)**.5
        
        trfig = plotTraces(regions,indices=ix, showCol=[showCol], offset=offset, smooth=smooth)#,time=regions.showTime[colToshow], showCol=showCol)
        trfig.update_yaxes(
            showticklabels=False,
            tickvals=[j*offset for j in range(len(ix))]
                          )
#         trfig.layout.update()
#         if narrowHover:
#         trfig.update_layout({"width":220,"height":200})
        return trfig
    except:
        return no_update


In [161]:
@app.callback(
    Output("raster", "figure"),
    [
     Input("filter_slider_feedback", "value"),
     Input("smooth_slider", "value")
    ])
def raster_callback(filterScale, smooth=0):
    from dash import no_update
    try:
        global regions
        timeScale, unit = filterScale.split()[:2]
        timeScale = float(timeScale)
        if unit=="min":
            timeScale *= 60
        if unit=="ms":
            timeScale /= 1000
        
        try:
            regions.raster[timeScale]
        except:
            regions.calc_raster(timeScale, smooth=smooth)
        if smooth:
            k = "%g_%g"%(timeScale, smooth)
        else:
            k = timeScale
        raster = regions.raster[k].astype(np.uint0)
        fig = go.Figure(go.Heatmap(
            z=raster,
            showscale=False
#             hoverinfo="text",text=[[str(i)] for i in regions.df.index]
        ))
        fig.update_yaxes(title_text='roi id')
        fig.update_layout({
            "width":  600,
            "height": 400,
            "margin": dict(l=20, r=10, t=50, b=20),
        })
        fig.update_xaxes(showticklabels=False)
        return fig
    except:
        return no_update


In [162]:
@app.callback(
    Output("save_feedback", "children"),
    [Input("save_button", "n_clicks")],
    [State("save_file_append", "value"   ),
     State("analysis_folder","children"),
     State("filter_slider_feedback", "value")
    ]
             )
def save_callback(n_clicks,filename, outDir, filterScale):
    try:
        if n_clicks <= 0:
            return "saving can take a few seconds."
        global regions
        
        timeScale, unit = filterScale.split()[:2]
        timeScale = float(timeScale)
        if unit=="min":
            timeScale *= 60
        if unit=="ms":
            timeScale /= 1000
        showCol = "zScore_%g"%timeScale
        if showCol not in regions.df:
            regions.fast_filter_traces(timeScale)
        regions.showTime = regions.time
        feedback = saveRois(regions, outDir, filename, col=showCol)
        feedback = sum([[el,html.Br()] for el in feedback],[])
        return feedback
    except:
        return "ERROR: "+exc_info().__repr__()+"\n"+str((n_clicks,filename, outDir, filterScale))

In [19]:
# app.layout = html.Div(children=APP_LAYOUT, style={"font-family":"Arial"})
# app._repr_html_() 
# link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
# HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')

In [201]:
from caiman import load as cload

In [ ]:
cload(["/data/HI/"])